In [109]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from pathlib import Path
import sys 
import networkx as nx

from helper import rr, prep, var_dict, graph

%load_ext autoreload
%autoreload 2


sys.path.append('/Users/alina/Desktop/MIT/code/ADHD/MTA/helper')
from helper import rr, prep, var_dict

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [110]:


if Path('/Volumes/Samsung_T5/MIT/mta').exists():
    data_root =     '/Volumes/Samsung_T5/MIT/mta'
    data_derived = '/Volumes/Samsung_T5/MIT/mta/output/derived_data'
else: 
    data_root = '/Users/alina/Desktop/MIT/code/data'
    data_deried = '/Users/alina/Desktop/MIT/code/data/output/derived_data'
#os.listdir(data_root)

In [111]:

cbcl  = pd.read_csv(Path(data_root, 'cbcl01.txt'), delimiter='\t')
var_qst_dict = dict(zip(cbcl.columns, cbcl.iloc[0]))
var_qst_dict
cbcl = prep.get_nonNaN_cols(cbcl)
cbcl_set = prep.set_baseline_dtypes(cbcl, dropna=False)

cbcl =  prep.split_data_from_timepoints(cbcl)

/var/folders/hf/frc4nxb532j7gqr3_36l6kmh0000gn/T/ipykernel_9327/3622179146.py:1: DtypeWarning:

Columns (0,1,2,6,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,2

Conversion encountered a problem. Attempt to drop description line.
Success


In [112]:
var_qst_dict

{'collection_id': 'collection_id',
 'cbcl01_id': 'cbcl01_id',
 'dataset_id': 'dataset_id',
 'subjectkey': 'The NDAR Global Unique Identifier (GUID) for research subject',
 'src_subject_id': "Subject ID how it's defined in lab/project",
 'interview_date': 'Date on which the interview/genetic test/sampling/imaging/biospecimen was completed. MM/DD/YYYY',
 'interview_age': 'Age in months at the time of the interview/test/sampling/imaging.',
 'sex': 'Sex of subject at birth',
 'comments_misc': 'Miscellaneous comments on study, interview, methodology relevant to this form data',
 'sports_des1': 'Sport1 description',
 'sports_time1': 'Amount of time spent on sport1',
 'sports_well1': 'How well youth does sport1',
 'sports_des2': 'Sport2 description',
 'sports_time2': 'Amount of time spent on sport2',
 'sports_well2': 'How well youth does sport2',
 'sports_des3': 'Sport3 description',
 'sports_time3': 'Amount of time spent on sport3',
 'sports_well3': 'How well youth does sport3',
 'activities

In [113]:
cbcl

{'b':      collection_id cbcl01_id dataset_id        subjectkey src_subject_id  \
 1             2155      2158       9221  NDAR_INVXP413AM1          P1001   
 2             2155      2159       9221  NDAR_INVXP413AM1          P1001   
 5             2155      2162       9221  NDAR_INVFC713CVK          P1002   
 6             2155      2163       9221  NDAR_INVFC713CVK          P1002   
 11            2155      2168       9221  NDAR_INVFE961JZZ          P1003   
 ...            ...       ...        ...               ...            ...   
 4055          2155      6212       9221  NDAR_INVGB054KLF          P1866   
 4058          2155      6215       9221  NDAR_INVJW789CKY          P1867   
 4059          2155      6216       9221  NDAR_INVJW789CKY          P1867   
 4061          2155      6218       9221  NDAR_INVPE614HET          P1868   
 4062          2155      6219       9221  NDAR_INVPE614HET          P1868   
 
      interview_date interview_age sex sports_time1 sports_well1  ...

In [114]:
#cbcl = cbcl['14']

In [115]:
cbcl_b = cbcl['b']
cbcl_14 = cbcl['14']
cbcl_24 = cbcl['24']
cbcl_36 = cbcl['36']

In [116]:
baseline_columns = var_dict.get_baseline_vars()
subject_spec_cols = ['collection_id', 'cbcl01_id', 'dataset_id', 'subjectkey',
       'src_subject_id', 'interview_date', 'interview_age', 'sex', 'sjtyp', 'collection_title', 'assbdic', 'actbdic']
cbcl_raw_cores_col = list(cbcl_b.columns[161:189])
type(cbcl_raw_cores_col)

list

In [117]:
unique_cols= list(set(np.concatenate((baseline_columns, subject_spec_cols + cbcl_raw_cores_col))))

In [118]:
cbcl_qst_data = cbcl_set.copy()
cbcl_qst_data.drop(columns=unique_cols, inplace=True)

In [119]:
cbcl_qst_data.ins.dropna().shape

(51,)

In [120]:
cbcl

{'b':      collection_id cbcl01_id dataset_id        subjectkey src_subject_id  \
 1             2155      2158       9221  NDAR_INVXP413AM1          P1001   
 2             2155      2159       9221  NDAR_INVXP413AM1          P1001   
 5             2155      2162       9221  NDAR_INVFC713CVK          P1002   
 6             2155      2163       9221  NDAR_INVFC713CVK          P1002   
 11            2155      2168       9221  NDAR_INVFE961JZZ          P1003   
 ...            ...       ...        ...               ...            ...   
 4055          2155      6212       9221  NDAR_INVGB054KLF          P1866   
 4058          2155      6215       9221  NDAR_INVJW789CKY          P1867   
 4059          2155      6216       9221  NDAR_INVJW789CKY          P1867   
 4061          2155      6218       9221  NDAR_INVPE614HET          P1868   
 4062          2155      6219       9221  NDAR_INVPE614HET          P1868   
 
      interview_date interview_age sex sports_time1 sports_well1  ...

In [121]:

# var_qst_dict = dict(zip(cbcl_b.columns, cbcl_b.iloc[0]))
# var_qst_dict

In [122]:
cbcl_b_set = cbcl_b.copy()
cbcl_b_set.drop(columns=unique_cols, inplace=True)

cbcl_14_set = cbcl_14.copy()
cbcl_14_set.drop(columns=unique_cols, inplace=True)

cbcl_24_set = cbcl_24.copy()
cbcl_24_set.drop(columns=unique_cols, inplace=True)

cbcl_36_set = cbcl_36.copy()
cbcl_36_set.drop(columns=unique_cols, inplace=True)

In [123]:
# df = cbcl_b_set
# question_dict = dict(zip(cbcl_b.columns, cbcl_b.iloc[0]))
# question_dict

In [124]:
#question_dict = {key: value.split('/')[0].strip() for key, value in question_dict.items()}

In [125]:
G_b03 = graph.compute_graph(cbcl_b_set, 'correlation', 0.3)
G_b04 = graph.compute_graph(cbcl_b_set, 'correlation', 0.4)
G_b05 = graph.compute_graph(cbcl_b_set, 'correlation', 0.5)

#G_14 = graph.compute_graph(cbcl_14_set, 'correlation', 0.3)
##G_24 = graph.compute_graph(cbcl_24_set, 'correlation', 0.3)
#G_36 = graph.compute_graph(cbcl_36_set, 'correlation', 0.3)

In [126]:
graph.plot_graph(G_b03, var_qst_dict, static=True, corr_width=False, plot_title="CBCL Baseline, Correlation threshold = 0.3")

In [127]:
graph.plot_graph(G_b04, var_qst_dict, static=True, corr_width=False, plot_title="CBCL Baseline, Correlation threshold = 0.4")

In [128]:
graph.plot_graph(G_b05, var_qst_dict, static=True, corr_width=False, plot_title="Baseline, Correlation threshold = 0.5")

In [129]:
#graph.plot_graph(G_14, question_dict, static=True, corr_width=False, plot_title="14 months")

In [130]:
#graph.plot_graph(G_24, question_dict, static=True, corr_width=False, plot_title="24 months")

In [131]:
#graph.plot_graph(G_36, question_dict, static=True, corr_width=False, plot_title="36 months")

In [132]:
G_neg = graph.compute_graph(cbcl_b_set, 'correlation', 0.35, allow_neg_edge=True)

In [133]:
graph.plot_graph(G_neg, var_qst_dict, neg_edges=True, static=True, plot_title="Baseline, Correlation threshold = 0.35")

In [134]:
snap = pd.read_csv(Path(data_root, 'snap01.txt'), delimiter="\t")
question_dict_snap = dict(zip(snap.columns, snap.iloc[0]))
snap_red_  = prep.get_nonNaN_cols(snap)

print(snap_red_.columns)

Index(['collection_id', 'snap01_id', 'dataset_id', 'subjectkey',
       'src_subject_id', 'interview_date', 'interview_age', 'sex',
       'snap_adhd_1', 'snap_adhd_2', 'snap_adhd_3', 'snap_adhd_4',
       'snap_adhd_5', 'snap_adhd_6', 'snap_adhd_7', 'snap_adhd_8',
       'snap_adhd_9', 'snap_adhd_11', 'snap_adhd_12', 'snap_adhd_13',
       'snap_adhd_14', 'snap_adhd_15', 'snap_adhd_16', 'snap_adhd_17',
       'snap_adhd_18', 'snap_adhd_19', 'snap_inattn_totalscore',
       'snap_inattn_avg', 'snap_hyp_totalscore', 'snap_hyp_avg', 'sna19',
       'sna20', 'sna21', 'sna22', 'sna23', 'sna24', 'sna25', 'sna26', 'sna27',
       'sna28', 'sna29', 'sna30', 'sna31', 'sna32', 'sna33', 'sna34', 'sna35',
       'sna36', 'sna37', 'sna38', 'sna39', 'snainatx', 'snainatt', 'snahypax',
       'snahypat', 'snaimpux', 'snaimput', 'snaaddwx', 'snaaddwt', 'snaoddx',
       'snaoddt', 'snatotlx', 'snatotlt', 'snahyimx', 'snaadhdx', 'snaadhdt',
       'snahix', 'snahit', 'snadbdx', 'snaxrsp', 'days_baseli

/var/folders/hf/frc4nxb532j7gqr3_36l6kmh0000gn/T/ipykernel_9327/2061752221.py:1: DtypeWarning:

Columns (0,1,2,6,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,76,77,78,79,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158) have mixed types. Specify dtype option on import or set low_memory=False.



In [135]:

cols2drop = [col for col in unique_cols if col in snap_red_.columns and col != 'days_baseline']
cols2drop.append('snap01_id')
cols2drop.append('version_form')
snap_raw_score_cols = snap.columns[55:74]
cols2drop = np.concatenate((cols2drop, snap_raw_score_cols, ['snap_inattn_avg', 'snap_hyp_totalscore', 'snap_hyp_avg']))
print(cols2drop)
snap_red = snap_red_.drop(columns=cols2drop)

#snap_red = snap[snap_cols]

['relationship' 'sjtyp' 'collection_id' 'subjectkey' 'site'
 'src_subject_id' 'sex' 'interview_age' 'assbdic' 'collection_title'
 'dataset_id' 'interview_date' 'actbdic' 'snap01_id' 'version_form'
 'snainatx' 'snainatt' 'snahypax' 'snahypat' 'snaimpux' 'snaimput'
 'snaaddwx' 'snaaddwt' 'snaoddx' 'snaoddt' 'snatotlx' 'snatotlt'
 'snahyimx' 'snaadhdx' 'snaadhdt' 'snahix' 'snahit' 'snadbdx' 'snaxrsp'
 'snap_inattn_avg' 'snap_hyp_totalscore' 'snap_hyp_avg']


In [140]:
snap  = prep.split_data_from_timepoints(snap_red)
snap_b = snap['b']
snap14 = snap['14']
snap24 = snap['24']
snap36 = snap['36']

In [141]:
question_dict_snap = dict(zip(snap_red.columns, snap_red.iloc[0]))

In [142]:
#snap_b = snap_b.drop(columns='days_baseline')

In [143]:
G_snap_b03 = graph.compute_graph(snap_b, 'correlation', 0.3)
graph.plot_graph(G_snap_b03, question_dict_snap, static=True, corr_width=True,plot_title="SNAP Baseline, Correlation threshold = 0.3")

In [144]:
G_snap_b073 = graph.compute_graph(snap_b, 'correlation', 0.73)
graph.plot_graph(G_snap_b073, question_dict_snap, static=True, corr_width=True,plot_title="SNAP Baseline, Correlation threshold = 0.73")

In [145]:
G_snap_14 = graph.compute_graph(snap14, 'correlation', 0.73)
graph.plot_graph(G_snap_14, question_dict_snap, static=True, corr_width=True)

In [146]:
G_snap_24 = graph.compute_graph(snap24, 'correlation', 0.73)
graph.plot_graph(G_snap_24, question_dict_snap, static=True, corr_width=True)

In [147]:
G_snap_36 = graph.compute_graph(snap36, 'correlation', 0.73)
graph.plot_graph(G_snap_36, question_dict_snap, static=True, corr_width=True)

In [ ]:
prep.print_nonNaN_shapes(snap)

0 collection_id collection_id (14581,)
1 snap01_id snap01_id (14581,)
2 dataset_id dataset_id (14581,)
3 subjectkey The NDAR Global Unique Identifier (GUID) for research subject (14581,)
4 src_subject_id Subject ID how it's defined in lab/project (14581,)
5 interview_date Date on which the interview/genetic test/sampling/imaging/biospecimen was completed. MM/DD/YYYY (14581,)
6 interview_age Age in months at the time of the interview/test/sampling/imaging. (14581,)
7 sex Sex of subject at birth (14581,)
10 snap_adhd_1 1. Often fails to give close attention to details or makes careless mistakes in schoolwork or tasks (14534,)
11 snap_adhd_2 2. Often has difficulty sustaining attention in tasks or play activities (14531,)
12 snap_adhd_3 3. Often does not seem to listen when spoken to directly (14539,)
13 snap_adhd_4 4. Often does not follow through on instructions and fails to finish schoolwork, chores, or duties (14533,)
14 snap_adhd_5 5. Often has difficulty organizing tasks and activit